# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazacorte,28.6290,-17.9293,16.54,77,40,4.63,ES,1705731538
1,1,fale old settlement,-9.3852,-171.2468,29.28,71,69,5.34,TK,1705731544
2,2,as sulayyil,20.4607,45.5779,17.14,32,22,5.28,SA,1705731547
3,3,mount pearl,47.5166,-52.7813,-3.25,84,100,3.09,CA,1705731553
4,4,howard springs,-12.4970,131.0470,30.49,79,100,7.13,AU,1705731554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600, 
    size = "Humidity",
    scale = 0.7,
    color = "City"  
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[(city_data_df["Max Temp"] >21) & (city_data_df["Max Temp"] <30) & 
                               (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"]<=20), :]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,avarua,-21.2078,-159.7750,26.03,78,20,2.06,CK,1705731612
43,43,ciudad guayana,8.3533,-62.6528,24.73,89,6,2.30,VE,1705731662
49,49,tura,25.5198,90.2201,22.62,37,0,1.74,IN,1705731672
53,53,saint-pierre,-21.3393,55.4781,28.82,79,0,4.63,RE,1705731678
70,70,kailua-kona,19.6406,-155.9956,24.98,92,0,1.54,US,1705731705
78,78,port alfred,-33.5906,26.8910,21.21,76,6,3.58,ZA,1705731717
81,81,carutapera,-1.1950,-46.0200,25.14,91,20,2.83,BR,1705731722
85,85,bredasdorp,-34.5322,20.0403,24.83,49,0,1.17,ZA,1705731729
86,86,kingston,17.9970,-76.7936,24.53,84,20,1.54,JM,1705731731
96,96,lubang,13.8584,120.1240,27.21,73,6,4.89,PH,1705731745


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= narrowed_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
26,avarua,CK,-21.2078,-159.7750,78,
43,ciudad guayana,VE,8.3533,-62.6528,89,
49,tura,IN,25.5198,90.2201,37,
53,saint-pierre,RE,-21.3393,55.4781,79,
70,kailua-kona,US,19.6406,-155.9956,92,
78,port alfred,ZA,-33.5906,26.8910,76,
81,carutapera,BR,-1.1950,-46.0200,91,
85,bredasdorp,ZA,-34.5322,20.0403,49,
86,kingston,JM,17.9970,-76.7936,84,
96,lubang,PH,13.8584,120.1240,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
#Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params= params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
avarua - nearest hotel: Paradise Inn
ciudad guayana - nearest hotel: Hotel
tura - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
port alfred - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
kingston - nearest hotel: Aphrodite Hotel
lubang - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
mangrol - nearest hotel: No hotel found
ugoofaaru - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
anloga - nearest hotel: Hotel de White House
somerset east - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
hermanus - nearest hotel: Aloe guest house
san antonio de pale - nearest hotel: No hotel found
macau - nearest hotel: 富華粵海酒店 Fu Hua Guang Dong Hotel
morant bay - nearest hotel: Morant Villas
porto 

,City,Country,Lat,Lng,Humidity,Hotel Name
26,avarua,CK,-21.2078,-159.7750,78,Paradise Inn
43,ciudad guayana,VE,8.3533,-62.6528,89,Hotel
49,tura,IN,25.5198,90.2201,37,No hotel found
53,saint-pierre,RE,-21.3393,55.4781,79,Tropic Hotel
70,kailua-kona,US,19.6406,-155.9956,92,Kona Seaside Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
# Configure the map plot

map_plot_2 = hotel_df.hvplot.points("Lng", "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600, 
    size = "Humidity",
    color = "City",
    scale = 0.8,
    hover_cols = ["Hotel Name", "Country"] 
)
# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)